<a href="https://colab.research.google.com/github/IlinValery/modelTraining-pricetags/blob/master/turicreate_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение модели на собранных ранее данных

## Подключение turicreate на collab
- [Инструкция по подключению turicreate на collab](https://medium.com/skafosai/turi-create-and-google-colab-bb3f393dbbb8)

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
!nvidia-smi

Thu May  2 19:56:28 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    15W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install turicreate==5.4
# The wrong version of MXNET will be installed
!pip uninstall -y mxnet
# Install CUDA10-compatible version of mxnet 
!pip install mxnet-cu100==1.4.0.post0


     |████████████████████████████████| 87.4MB 121kB/s 
     |████████████████████████████████| 2.7MB 38.8MB/s 
     |████████████████████████████████| 23.8MB 2.7MB/s 
ERROR: mxnet 1.1.0.post0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.16.3 which is incompatible.
ERROR: mxnet 1.1.0.post0 has requirement requests<2.19.0,>=2.18.4, but you'll have requests 2.21.0 which is incompatible.
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Uninstalling mxnet-1.1.0.post0:
  Successfully uninstalled mxnet-1.1.0.post0
     |████████████████████████████████| 487.9MB 30kB/s 
     |████████████████████████████████| 13.8MB 36.6MB/s 
ERROR: turicreate 5.4 requires mxnet<1.2.0,>=1.1.0, which is not installed.
ERROR: spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompati

## Подключение библиотек

In [0]:
import turicreate as tc
from turicreate import SArray
import pandas as pd
import sys, os
# Tell turicreate to use ALL available GPUs
tc.config.set_num_gpus(-1)

         To fix this, please install the currently recommended version:

             pip uninstall -y mxnet && pip install mxnet==1.1.0

         If you want to use a CUDA GPU, then change 'mxnet' to 'mxnet-cu90' (adjust 'cu90' depending on your CUDA version):



## Подключение хранилища для доступа к sframe

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
source_folder = "/content/drive/My Drive/pricetags-detection/turicreate/"

In [0]:
data = tc.SFrame(source_folder+'annotations.sframe')

## Разделение данных и обучение модели

In [0]:
train_data, test_data = data.random_split(0.8)

In [0]:
model = tc.object_detector.create(train_data, max_iterations=1500)

Using 'image' as feature column
Using 'annotations' as annotations column
Download completed: /var/tmp/model_cache/darknet.params
Setting 'batch_size' to 32
Using GPU to create model (Tesla T4)
+--------------+--------------+--------------+
| Iteration    | Loss         | Elapsed Time |
+--------------+--------------+--------------+
| 1            | 9.555        | 44.1         |
| 2            | 9.609        | 68.8         |
| 3            | 9.577        | 91.4         |
| 4            | 9.644        | 114.0        |
| 5            | 9.680        | 137.1        |
| 6            | 9.740        | 160.7        |
| 7            | 9.709        | 183.0        |
| 8            | 9.714        | 207.2        |
| 9            | 9.759        | 238.5        |
| 10           | 9.743        | 267.7        |
| 11           | 9.764        | 298.7        |
| 12           | 9.714        | 335.2        |
| 13           | 9.616        | 361.7        |
| 14           | 9.533        | 386.7        |
| 15   

In [0]:
# Save predictions to an SArray
predictions = model.predict(test_data)

Predicting  1/95
Predicting 33/95
Predicting 65/95
Predicting 95/95


In [0]:
# Evaluate the model and save the results into a dictionary
metrics = model.evaluate(test_data)

Predicting  1/95
Predicting 33/95
Predicting 65/95
Predicting 95/95


In [0]:
# Save the model for later use in Turi Create
model.save(source_folder+'pricetags_1500iters.model')

In [0]:
# Export for use in Core ML
model.export_coreml(source_folder+'PricetagsDetector.mlmodel')

KeyError: ignored

## Далее используем файл turicreate-save_model.ipynb